In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
data = pd.read_csv('tweet_emotions.csv')

In [3]:

data.shape

(40000, 3)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
y = data['sentiment']

In [6]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK resources (only needed once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load your tabular data into a pandas DataFrame
# Assuming your data is in a CSV file, replace 'data.csv' with your file path
# data = pd.read_csv('data.csv')

# Define a function for text preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove punctuation and special characters
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Apply stemming or lemmatization (choose one)
    # Stemming reduces words to their root form (e.g., running -> run)
    ps = PorterStemmer()
    words = [ps.stem(word) for word in words]

    # Lemmatization reduces words to their base or dictionary form (e.g., running -> run)
    # lemma = WordNetLemmatizer()
    # words = [lemma.lemmatize(word) for word in words]

    # Join the words back into a single string
    preprocessed_text = ' '.join(words)
    return preprocessed_text

# Apply the preprocessing function to the 'text' column in the DataFrame
data['preprocessed_text'] = data['content'].apply(preprocess_text)

# Save the preprocessed data to a new CSV file (optional)
# data.to_csv('preprocessed_data.csv', index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
X = data['preprocessed_text']

In [31]:
tf_idf = TfidfVectorizer(max_features=20000)

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
np.unique(np.array(y), return_counts=True)

(array(['anger', 'boredom', 'empty', 'enthusiasm', 'fun', 'happiness',
        'hate', 'love', 'neutral', 'relief', 'sadness', 'surprise',
        'worry'], dtype=object),
 array([ 110,  179,  827,  759, 1776, 5209, 1323, 3842, 8638, 1526, 5165,
        2187, 8459]))

In [34]:
X_ = tf_idf.fit_transform(X).toarray()

In [35]:
X_.shape, y.shape

((40000, 20000), (40000,))

In [36]:
X_.shape, y.shape


((40000, 20000), (40000,))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_, y, test_size=0.25, random_state=42)

In [23]:

X_train.shape, X_val.shape, y_train.shape

((30000, 10000), (10000, 10000), (30000,))

In [24]:
# from sklearn.svm import SVC

In [25]:
# svm = SVC(kernel='rbf', gamma=0.001, C=0.01,random_state=42, probability=True).fit(X_idf, y_train)

In [27]:
inputs = tf.keras.Input(shape=(None, 30000, 10000))
l0 = tf.keras.layers.Dense(2048*2, activation='relu')(inputs)

l1 = tf.keras.layers.Dense(2048, activation='relu')(l0)
dr = tf.keras.layers.Dropout(0.02)(l1)
l2 = tf.keras.layers.Dense(1024, activation='relu')(dr)
l3 = tf.keras.layers.Dense(512, activation='relu')(l2)
l4 = tf.keras.layers.Dense(256, activation='relu')(l3)
l5 = tf.keras.layers.Dense(128, activation='relu')(l4)
out = tf.keras.layers.Dense(13, activation='softmax')(l5)

model = tf.keras.Model(inputs=inputs, outputs=out)





# inputs = tf.keras.Input(shape=(3,))
# x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
# outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
# model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [28]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_numeric = label_encoder.fit_transform(y_train)
y_val_numeric = label_encoder.transform(y_val)

In [29]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [30]:

num_epochs = 10
history = model.fit(X_train, y_train_numeric, epochs=num_epochs,
                    validation_data=(X_val, y_val_numeric))

Epoch 1/10
938/938 [==============================] - 22s 15ms/step - loss: 2.0396 - accuracy: 0.2801 - val_loss: 1.9953 - val_accuracy: 0.3263
Epoch 2/10
938/938 [==============================] - 15s 16ms/step - loss: 1.8022 - accuracy: 0.3833 - val_loss: 1.9945 - val_accuracy: 0.3286
Epoch 3/10
938/938 [==============================] - 13s 14ms/step - loss: 1.3396 - accuracy: 0.5528 - val_loss: 2.4682 - val_accuracy: 0.3041
Epoch 4/10
938/938 [==============================] - 13s 14ms/step - loss: 0.8297 - accuracy: 0.7234 - val_loss: 2.8946 - val_accuracy: 0.2654
Epoch 5/10
938/938 [==============================] - 13s 14ms/step - loss: 0.5217 - accuracy: 0.8300 - val_loss: 3.5164 - val_accuracy: 0.2671
Epoch 6/10
938/938 [==============================] - 13s 14ms/step - loss: 0.3483 - accuracy: 0.8893 - val_loss: 3.8398 - val_accuracy: 0.2678
Epoch 7/10
938/938 [==============================] - 13s 14ms/step - loss: 0.2443 - accuracy: 0.9236 - val_loss: 4.0374 - val_accuracy:

KeyboardInterrupt: ignored